In [1]:
import os
import requests

In [2]:
url = "https://pressbooks.oer.hawaii.edu/humannutrition2/open/download?type=pdf"
pdf_path = "human-nutrition-text.pdf"
if os.path.exists(pdf_path):
    print("[INFO] pdf file already exists")
else:
    print("[INFO] Downloading...")
    response = requests.get(url)
    if response.status_code == 200:
        with open(pdf_path, "wb") as f:
            f.write(response.content)
        print(f"[INFO] file have been downloaded sucessfully {pdf_path}")
    else:
        print(f"[INFO] can not download file {pdf_path}")

[INFO] Downloading...
[INFO] file have been downloaded sucessfully human-nutrition-text.pdf


In [3]:
!pip install fitz
!pip install tqdm
!pip install PyMuPDF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.6/95.6 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.5/421.5 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.9 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.8/30.8 MB 41.0 MB/s eta 0:00:00


In [4]:
import fitz
from tqdm.auto import tqdm

def get_pages_info(path):
    pdf_doc = fitz.open(path)
    page_and_text = []
    for page_number in tqdm(range(len(pdf_doc))):
        page = pdf_doc.load_page(page_number)
        content = page.get_text()
        content = content.replace("\n", " ").strip()
        page_and_text.append({
            "page_number": page_number - 41,
            "content": content,
            "words_per_page": len(content.split(" ")),
            "sentence_per_page": len(content.split(". ")),
            "chars_per_page": len(content),
            "tokens_per_page": len(content) // 4
        })
    pdf_doc.close()
    return page_and_text

In [5]:
pages_and_texts = get_pages_info(pdf_path)

  0%|          | 0/1208 [00:00<?, ?it/s]

In [6]:
pages_and_texts[46]

{'page_number': 5,
 'content': 'Macronutrients  Nutrients  that  are  needed  in  large  amounts  are  called  macronutrients. There are three classes of macronutrients:  carbohydrates, lipids, and proteins. These can be metabolically  processed into cellular energy. The energy from macronutrients  comes from their chemical bonds. This chemical energy is  converted into cellular energy that is then utilized to perform work,  allowing our bodies to conduct their basic functions. A unit of  measurement of food energy is the calorie. On nutrition food labels  the amount given for “calories” is actually equivalent to each calorie  multiplied by one thousand. A kilocalorie (one thousand calories,  denoted with a small “c”) is synonymous with the “Calorie” (with a  capital “C”) on nutrition food labels. Water is also a macronutrient in  the sense that you require a large amount of it, but unlike the other  macronutrients, it does not yield calories.  Carbohydrates  Carbohydrates are molecule

In [7]:
import pandas as pd
df = pd.DataFrame(pages_and_texts)
df.tail()

,page_number,content,words_per_page,sentence_per_page,chars_per_page,tokens_per_page
1203,1162,39. Exercise 10.2 & 11.3 reused “Egg Oval Food...,252,18,1676,419
1204,1163,Images / Pixabay License; “Pumpkin Cartoon Ora...,254,10,1617,404
1205,1164,Flashcard Images Note: Most images in the fla...,261,13,1715,428
1206,1165,ShareAlike 11. Organs reused “Pancreas Organ ...,268,13,1733,433
1207,1166,23. Vitamin D reused “The Functions of Vitamin...,44,3,257,64


In [8]:
df.describe().round(2)

,page_number,words_per_page,sentence_per_page,chars_per_page,tokens_per_page
count,1208.00,1208.00,1208.00,1208.00,1208.00
mean,562.50,198.89,9.97,1148.59,286.79
std,348.86,95.75,6.19,560.44,140.09
min,-41.00,1.00,1.00,0.00,0.00
25%,260.75,134.00,4.00,762.75,190.00
50%,562.50,215.00,10.00,1232.50,307.50
75%,864.25,271.25,14.00,1605.25,401.00
max,1166.00,429.00,32.00,2308.00,577.00


In [9]:
!pip install nltk

In [10]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
import copy
def get_chunks(text, chunk_size=10):
    sentences = sent_tokenize(text)
    chunks = [sentences[i: i+chunk_size] for i in range(0, len(sentences), chunk_size)]
    return chunks

def add_chunk(pages_and_texts):
    res = copy.deepcopy(pages_and_texts)
    for page_number in tqdm(range(len(pages_and_texts))):
        content = res[page_number]['content']
        chunks = get_chunks(content)
        res[page_number]['chunked_sentence'] = chunks
        res[page_number]['num_chunk'] = len(chunks)
    return res

pages_and_chunks = add_chunk(pages_and_texts)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


  0%|          | 0/1208 [00:00<?, ?it/s]

In [11]:
pages_and_chunks[49]

{'page_number': 8,
 'content': 'Water  There is one other nutrient that we must have in large quantities:  water. Water does not contain carbon, but is composed of two  hydrogens and one oxygen per molecule of water. More than 60  percent of your total body weight is water. Without it, nothing could  be transported in or out of the body, chemical reactions would not  occur, organs would not be cushioned, and body temperature would  fluctuate widely. On average, an adult consumes just over two liters  of water per day from food and drink combined. Since water is so  critical for life’s basic processes, the amount of water input and  output is supremely important, a topic we will explore in detail in  Chapter 4.  Micronutrients  Micronutrients are nutrients required by the body in lesser  amounts, but are still essential for carrying out bodily functions.  Micronutrients include all the essential minerals and vitamins.  There are sixteen essential minerals and thirteen vitamins (See  Tab

In [12]:
df = pd.DataFrame(pages_and_chunks)
df.head()

,page_number,content,words_per_page,sentence_per_page,chars_per_page,tokens_per_page,chunked_sentence,num_chunk
0,-41,Human Nutrition: 2020 Edition,4,1,29,7,[[Human Nutrition: 2020 Edition]],1
1,-40,,1,1,0,0,[],0
2,-39,Human Nutrition: 2020 Edition UNIVERSITY OF ...,54,1,320,80,[[Human Nutrition: 2020 Edition UNIVERSITY O...,1
3,-38,Human Nutrition: 2020 Edition by University of...,32,1,212,53,[[Human Nutrition: 2020 Edition by University ...,1
4,-37,Contents Preface University of Hawai‘i at Mā...,145,2,797,199,[[Contents Preface University of Hawai‘i at ...,1


In [13]:
df.describe().round(2)

,page_number,words_per_page,sentence_per_page,chars_per_page,tokens_per_page,num_chunk
count,1208.00,1208.00,1208.00,1208.00,1208.00,1208.00
mean,562.50,198.89,9.97,1148.59,286.79,1.52
std,348.86,95.75,6.19,560.44,140.09,0.64
min,-41.00,1.00,1.00,0.00,0.00,0.00
25%,260.75,134.00,4.00,762.75,190.00,1.00
50%,562.50,215.00,10.00,1232.50,307.50,1.00
75%,864.25,271.25,14.00,1605.25,401.00,2.00
max,1166.00,429.00,32.00,2308.00,577.00,3.00


In [14]:
import re
def create_page_chunk(pages_and_chunks):
    page_chunk_list = []
    for page_num in tqdm(range(len(pages_and_chunks))):
        for chunk in pages_and_chunks[page_num]["chunked_sentence"]:
            dict = {}
            dict["page_num"] = pages_and_chunks[page_num]["page_number"]
            item = re.sub(r'\.([A-Z])', r'. \1', ''.join(chunk))
            dict["chunk"] = item
            dict["num_token_per_chunk"] = len(item) / 4
            page_chunk_list.append(dict)
    return page_chunk_list

In [15]:
temp = ['Effects of Alcohol Abuse on the Brain  A small amount (up to 10%) of the liver acetaldehyde may  accumulate inside the liver cells.',
 'As more alcohol is ingested, this  stimulates the production of acetaldehyde by both the alcohol  dehydrogenase and MEOS systems.',
 'As the levels of acetaldehyde  increase inside the liver cells with heavy consumption of alcohol,  some of the acetaldehyde diffuse into the blood circulation.',
 'In  circulation, high levels of acetaldehyde cause nausea and vomiting.',
 'Vomiting causes more body dehydration and loss of electrolytes.',
 'If the dehydration becomes severe enough, this can impair brain  function and a person may lose consciousness.',
 'Alcohol can adversely affect nearly every area of the brain.',
 'When  BAC rises, the central nervous system is depressed.',
 'Alcohol disrupts  the way nerve cells communicate with each other by interfering  with receptors on certain cells.',
 'The immediate impact of alcohol  on the brain can be seen in the awkwardly displayed symptoms  of confusion, blurred vision, slurred speech, and other signs of  intoxication.']

import re
pattern = r'\.([A-Z])'
repl = r'. \1'
len(re.sub(pattern, repl, ''.join(temp))) / 4

269.25

In [16]:
page_chunk = create_page_chunk(pages_and_chunks)

  0%|          | 0/1208 [00:00<?, ?it/s]

In [17]:
df = pd.DataFrame(page_chunk)
df.head()

,page_num,chunk,num_token_per_chunk
0,-41,Human Nutrition: 2020 Edition,7.25
1,-39,Human Nutrition: 2020 Edition UNIVERSITY OF ...,80.00
2,-38,Human Nutrition: 2020 Edition by University of...,53.00
3,-37,Contents Preface University of Hawai‘i at Mā...,199.25
4,-36,Lifestyles and Nutrition University of Hawai‘...,244.00


In [18]:
df.describe()

,page_num,num_token_per_chunk
count,1840.000000,1840.000000
mean,585.333152,187.685326
std,347.485864,114.466212
min,-41.000000,3.500000
25%,283.000000,80.250000
50%,589.500000,190.500000
75%,891.000000,284.812500
max,1166.000000,468.500000


In [19]:
new_df = df[df["num_token_per_chunk"] >= 30]

In [20]:
import random
page_chunk_over30 = new_df.to_dict(orient="records")
random.sample(page_chunk_over30, 1)

[{'page_num': 301,
  'chunk': 'The Role of Lipids in Food  UNIVERSITY OF HAWAI‘I AT MĀNOA FOOD SCIENCE AND HUMAN  NUTRITION PROGRAM AND HUMAN NUTRITION PROGRAM  High Energy Source  Fat-rich foods naturally have a high caloric density. Foods that are  high in fat contain more calories than foods high in protein or  carbohydrates. As a result, high-fat foods are a convenient source  of energy. For example, 1 gram of fat or oil provides 9 kilocalories  of energy, compared with 4 kilocalories found in 1 gram of  carbohydrate or protein. Depending on the level of physical activity  and on nutritional needs, fat requirements vary greatly from person  to person. When energy needs are high, the body welcomes the  high-caloric density of fats. For instance, infants and growing  children require proper amounts of fat to support normal growth  and development. If an infant or child is given a low-fat diet for  an extended period, growth and development will not progress  normally. Other individua

In [21]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.1/163.1 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 54.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 66.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 88.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [22]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("all-mpnet-base-v2")

sentences = [
    "This framework generates embeddings for each input sentence",
    "Sentences are passed as a list of string.",
    "The quick brown fox jumps over the lazy dog."
]

embeddings = model.encode(sentences)
embeddings.shape

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

(3, 768)

In [23]:
# start = timer()
# for item in tqdm(page_chunk_over30):
#   item['embedding'] = model.encode(item['chunk'])
# end = timer()
# print(end-start)

In [24]:
!nvidia-smi

Sun Mar 24 16:04:47 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0              26W /  70W |    603MiB / 15360MiB |      5%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [26]:
import torch
from time import perf_counter as timer
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model.to(device)
start = timer()
for item in tqdm(page_chunk_over30):
  item['embedding'] = model.encode(item['chunk'], convert_to_tensor=True)
end = timer()
print(end-start)

  0%|          | 0/1676 [00:00<?, ?it/s]

36.26210109600004


In [27]:
text_chunks = [item['chunk'] for item in page_chunk_over30]
start = timer()
embeddings = model.encode(text_chunks, batch_size=32, convert_to_tensor=True)
end = timer()
print(end-start)

23.007503765000024


In [28]:
embeddings.shape

torch.Size([1676, 768])

In [29]:
embeddings.dtype

torch.float32

In [49]:
import textwrap
from sentence_transformers import util

def wrap_text(text, w=80):
  formated = textwrap.wrap(text, w)
  return '\n'.join(formated)

def get_top(query, context):
  embeded_query = model.encode(query, convert_to_tensor=True).unsqueeze(0)
  scores = util.dot_score(embeded_query, context)
  score, index = torch.topk(scores, k=5, dim=1)
  return score, index

def get_ans(query, context):
  scores, indexes = get_top(query, context)
  print(f"QUERY: {query}")
  for score, index in zip(scores[0], indexes[0]):
    formated = wrap_text(page_chunk_over30[index]['chunk'])
    print(f"SCORE: {score}\nTEXT: \n{formated}")

query = "good foods for protein"
get_ans(query, embeddings)

QUERY: good foods for protein
SCORE: 0.772891104221344
TEXT: 
Dietary Sources of Protein  The protein food group consists of foods made from
meat, seafood,  poultry, eggs, soy, dry beans, peas, and seeds. According to the
Harvard School of Public Health, “animal protein and vegetable  protein probably
have the same effects on health. It’s the protein  package that’s likely to make
a difference.”1  1. Protein: The Bottom Line. Harvard School of Public
Proteins, Diet, and Personal Choices  |  411
SCORE: 0.7647285461425781
TEXT: 
Additionally, a person should consume 8 ounces of cooked seafood  every week
(typically as two 4-ounce servings) to assure they are  getting the healthy
omega-3 fatty acids that have been linked to a  lower risk for heart disease.
Another tip is choosing to eat dry beans,  peas, or soy products as a main dish.
Some of the menu choices  include chili with kidney and pinto beans, hummus on
pita bread,  and black bean enchiladas. You could also enjoy nuts in a varie

In [31]:
query = torch.tensor([1,2,3]).unsqueeze(0)
context = torch.tensor([[1,2,3],[4,5,6],[-1,-2,-3]])
scores = util.dot_score(query, context)
torch.topk(scores, k=2, dim=1)


torch.return_types.topk(
values=tensor([[32, 14]]),
indices=tensor([[1, 0]]))

In [32]:
gpu_memory_bytes = torch.cuda.get_device_properties(0).total_memory
gpu_memory_gb = round(gpu_memory_bytes / (2**30))
print(gpu_memory_gb)

15


In [33]:
if gpu_memory_gb < 5.1:
    print(f"Your available GPU memory is {gpu_memory_gb}GB, you may not have enough memory to run a Gemma LLM locally without quantization.")
elif gpu_memory_gb < 8.1:
    print(f"GPU memory: {gpu_memory_gb} | Recommended model: Gemma 2B in 4-bit precision.")
    use_quantization_config = True
    model_id = "google/gemma-2b-it"
elif gpu_memory_gb < 19.0:
    print(f"GPU memory: {gpu_memory_gb} | Recommended model: Gemma 2B in float16 or Gemma 7B in 4-bit precision.")
    use_quantization_config = False
    model_id = "google/gemma-2b-it"
elif gpu_memory_gb > 19.0:
    print(f"GPU memory: {gpu_memory_gb} | Recommend model: Gemma 7B in 4-bit or float16 precision.")
    use_quantization_config = False
    model_id = "google/gemma-7b-it"

print(f"use_quantization_config set to: {use_quantization_config}")
print(f"model_id set to: {model_id}")

GPU memory: 15 | Recommended model: Gemma 2B in float16 or Gemma 7B in 4-bit precision.
use_quantization_config set to: False
model_id set to: google/gemma-2b-it


In [34]:
!pip install -U "huggingface_hub[cli]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.4/346.4 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 9.3 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [35]:
!pip install bitsandbytes
!pip install transformers
!pip install accelerate
!pip install -i https://pypi.org/simple/ bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 4.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple/


In [36]:
from huggingface_hub import notebook_login
notebook_login()

In [38]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
quantization_config = BitsAndBytesConfig(load_in_4bit=True,
                                         bnb_4bit_compute_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")
llm_model = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it", torch_dtype=torch.float16,
                                                 low_cpu_mem_usage=False, # use as much memory as we can
                                                 attn_implementation="sdpa")

tokenizer_config.json:   0%|          | 0.00/2.16k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/888 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [39]:
llm_model

GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): GemmaRotaryEmbedding()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (up_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (down_proj): Linear(in_features=16384, out_features=2048, bias=False)
          (act_fn): GELUActivation()
        )
        (input_layernorm): GemmaRMSNorm()
        (post_attention_layernorm): GemmaRMSNorm()
      )
    )
    (norm): GemmaRM

In [40]:
torch.cuda.get_device_capability(0)[0]

7

In [41]:
def num_params(model):
  return sum([p.numel() for p in model.parameters()])
num_params(llm_model)

2506172416

In [42]:
input_text = "What are the macronutrients and what are their functions in the body?"
chat = [
    {"role" : "user", "content": input_text}
]
tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)

'<bos><start_of_turn>user\nWhat are the macronutrients and what are their functions in the body?<end_of_turn>\n<start_of_turn>model\n'

In [43]:
tokenizer

GemmaTokenizerFast(name_or_path='google/gemma-2b-it', vocab_size=256000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<bos>', 'eos_token': '<eos>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<start_of_turn>', '<end_of_turn>']}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<eos>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("<bos>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	106: AddedToken("<start_of_turn>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	107: AddedToken("<end_of_turn>", rstrip=False, lstr

In [44]:
llm_model = llm_model.to(device)
encoded_input = tokenizer(input_text, return_tensors='pt').to(device)

In [45]:
outputs = llm_model.generate(**encoded_input, max_new_tokens=256)
print(f"{tokenizer.decode(outputs[0])}")

<bos>What are the macronutrients and what are their functions in the body?

Sure, here's a breakdown of the macronutrients and their functions:

**Macronutrients:**

* **Carbohydrates:**
    * Provide energy for the body's cells and tissues.
    * Carbohydrates are the primary source of energy for most cells.
    * Complex carbohydrates include starches, fibers, and sugars.
* **Proteins:**
    * Build and repair tissues, enzymes, and hormones.
    * Proteins are essential for muscle growth and repair.
    * Essential amino acids are needed for protein synthesis.
* **Fats:**
    * Provide energy, insulation, and hormone production.
    * Healthy fats include olive oil, avocado, nuts, and seeds.
    * Trans fats should be avoided as they can raise cholesterol levels.

**Functions of Macronutrients:**

* **Energy:** Macronutrients provide the energy needed for various bodily functions, including muscle contraction, nerve signaling, and metabolic processes.
* **Building and Repair:** Prote

In [46]:
gpt4_questions = [
    "What are the macronutrients, and what roles do they play in the human body?",
    "How do vitamins and minerals differ in their roles and importance for health?",
    "Describe the process of digestion and absorption of nutrients in the human body.",
    "What role does fibre play in digestion? Name five fibre containing foods.",
    "Explain the concept of energy balance and its importance in weight management."
]

# Manually created question list
manual_questions = [
    "How often should infants be breastfed?",
    "What are symptoms of pellagra?",
    "How does saliva help with digestion?",
    "What is the RDI for protein per day?",
    "water soluble vitamins"
]

query_list = gpt4_questions + manual_questions
query_list

['What are the macronutrients, and what roles do they play in the human body?',
 'How do vitamins and minerals differ in their roles and importance for health?',
 'Describe the process of digestion and absorption of nutrients in the human body.',
 'What role does fibre play in digestion? Name five fibre containing foods.',
 'Explain the concept of energy balance and its importance in weight management.',
 'How often should infants be breastfed?',
 'What are symptoms of pellagra?',
 'How does saliva help with digestion?',
 'What is the RDI for protein per day?',
 'water soluble vitamins']

In [99]:
def ask(query):
  scores, indices = get_top(query, embeddings)
  items = []
  for score, index in zip(scores[0], indices[0]):
    items.append(page_chunk_over30[index]['chunk'])
  ctx = "- " + "\n- ".join(items)

  base_prompt = """Remember to use the following context items to answer the user query, try to find keyword in context to answer, the answer can be a litle bit creative:
{ctx}\n
User query: {query}
Answer:"""
  base_prompt = base_prompt.format(ctx=ctx, query=query)
  chat = [
    {"role" : "user", "content": base_prompt}
  ]
  prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
  encoded_input = tokenizer(prompt, return_tensors='pt').to(device)
  outputs = llm_model.generate(**encoded_input, max_new_tokens=256, temperature=0.3, do_sample=True)
  print(f"{tokenizer.decode(outputs[0])}")

query = random.sample(query_list, k=1)[0]
ask(query)


<bos><bos><start_of_turn>user
Remember to use the following context items to answer the user query, try to find keyword in context to answer, the answer can be a litle bit creative:
- recommended that users complete these activities using a  desktop or laptop computer and in Google Chrome. An interactive or media element has been  excluded from this version of the text. You can  view it online here:  http:/ /pressbooks.oer.hawaii.edu/ humannutrition2/?p=347    592  |  Water-Soluble Vitamins
- Water-Soluble Vitamins  UNIVERSITY OF HAWAI‘I AT MĀNOA FOOD SCIENCE AND HUMAN  NUTRITION PROGRAM AND HUMAN NUTRITION PROGRAM  All water-soluble vitamins play a different kind of role in energy  metabolism; they are required as functional parts of enzymes  involved in energy release and storage. Vitamins and minerals that  make up part of enzymes are referred to as coenzymes and  cofactors, respectively. Coenzymes and cofactors are required by  enzymes to catalyze a specific reaction. They assist i

In [101]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install pipreqs

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.3/798.3 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.4/257.4 kB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 60.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.0/48.0 kB 7.2 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=ba0d521e923c20ca5a1902bd5d1e0413f923a271dc576d309fcd1dbf9f77f468
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt
  Attempting uninstall: mistune
    Found existing installation: mistune 0.8.4
    Uninstalling mistune-0.8.4:
      Successfully uninstalled mistune-0.8.4
  Attempting uninstall: ipython
    Found existing installation: ipython 7.34.0
    Uninstalling ipython-7.34.0:
      Successfully uninstalled ipython-7.34.0
  Attempting uninstall: nbconver

In [103]:
!pipreqs /content

INFO: Not scanning for jupyter notebooks.
INFO: Successfully saved requirements file in /content/requirements.txt
